In [5]:
import torch
import torch.utils.benchmark as benchmark

from alibi_generator import ALiBiAttentionMaskGenerator
from metaseq import buffered_future_mask

In [6]:
seq_len = 512
batch_size = 1

# The num_heads is hard coded inside buffered_future_mask as 8
# Metaseq's buffered_future_mask is a class method, num_heads is 
# taken from a class attribute.
t0 = benchmark.Timer(
    stmt = "buffered_future_mask(x)",
    globals = {
        "buffered_future_mask": buffered_future_mask,
        "x": torch.empty((seq_len, batch_size))
        },
    label = "Metaseq")


# model_size is abitrary for alibi,
# but it's the expected dimension for X in fairseq's 2 AttentionMask
model_size = 1

t1 = benchmark.Timer(
    stmt = """
        alibi_gen = ALiBiAttentionMaskGenerator(num_heads=8)
        alibi_gen(x)
        """,
    globals = {
        "ALiBiAttentionMaskGenerator": ALiBiAttentionMaskGenerator,
        "x": torch.empty((seq_len, model_size))
        },
    label = "Fairseq2")

In [7]:
res0 = t0.timeit(30000)
res1 = t1.timeit(30000)
print(res0)
print(res1)

Metaseq
  756.01 us
  1 measurement, 30000 runs , 1 thread
Fairseq2
  617.52 us
  1 measurement, 30000 runs , 1 thread
